In [133]:
import pandas as pd
import geopandas as gpd
import fiona

In [10]:
url_311 = {  
2019: "https://opendata.arcgis.com/datasets/80ac366a2a954cdc890bb8728b698c27_0.geojson",
2018: "https://opendata.arcgis.com/datasets/3dd40fe4a3084021a06c01208e6b0c60_0.geojson",
2017: "https://opendata.arcgis.com/datasets/f978834f53bc43e88a294bb404360a64_0.geojson",
2016: "https://opendata.arcgis.com/datasets/9ec6339588bf433589b6ea77a9a01514_0.geojson",
2015: "https://opendata.arcgis.com/datasets/b2fad8490dcb43b18185c380c8caf4cc_0.geojson",
2014: "https://opendata.arcgis.com/datasets/b5d2dc0952594afcabcd0f84bceef1d8_0.geojson"
}

In [37]:
def merge_311(url_311):
    df_dict={}
    for i in url_311.keys():
        df = gpd.read_file(url_311.get(i))
        df_dict.update({i:df})
    return(df_dict)

In [38]:
df_dict_311=merge_311(url_311)

In [156]:
df=gpd.GeoDataFrame(pd.concat(list(df_dict_311.values()), 
                              ignore_index=True, 
                              sort=True), 
                    crs=list(df_dict_311.values())[0].crs)

In [157]:
df.dtypes

CASEID               int64
CASESTATUS           int64
CLOSEDDATETIME      object
ENQUIRYTYPEID      float64
LastUpdateDate      object
OBJECTID             int64
OPENEDDATETIME      object
REASONNAME          object
SUBJECTNAME         object
TITLE               object
TYPENAME            object
XCOORD             float64
YCOORD             float64
geometry          geometry
dtype: object

In [158]:
df.REASONNAME.value_counts()

Traffic Control                    106162
Private Property                    60447
Snow & Ice                          39582
Animal Control                      28091
Graffiti / Clean City               27020
Street Maintenance                  20382
Unhealthy Conditions                10677
Traffic                              7240
Lighting                             6802
Complaints                           5648
Garbage & Recycling                  4941
Sewer & Storm Drain                  3341
Commercial / Business Licensing      3205
Biking                               2486
Construction Inspections             2329
Street Cleaning / Sweeping           1956
Commercial Property                  1801
Fire                                 1769
Inspection                           1614
Parking                              1414
Police                               1127
Water                                1064
Traffic Safety                        485
Zoning                            

In [159]:
pd.set_option('display.max_rows', 150)
sorted(df.TYPENAME.unique())

['Abandoned Vehicle',
 'Abandoned Vehicle - Open311',
 'Abandoned/Illegally Parked Bicycle',
 'Air Pollution Complaint',
 'Animal Complaint - Livability',
 'Animal Complaint - Public Health',
 'Barricade Pickup / Placement',
 'Bicycle Lane Issue',
 'Bicycle Trail Improvement Request',
 'Bike Lane Issue - Open311',
 'Bridge Complaint',
 'Campaign Sign Complaint',
 'Certificate of Code Compliance Request',
 'Certificate of Occupancy Request',
 'Chemical Spill',
 'Commercial Disability Zone',
 'Commercial Food Safety/Sanitation Complaint',
 'Commercial Structure Unsound',
 'Commercial/Mixed Conditions Complaint',
 'Debris Alley/Street - Open311',
 'Debris in the Street or Alley',
 'Emergency Pollution Complaint',
 'Exterior Nuisance - Self Service',
 'Exterior Nuisance Complaint',
 'Fence or Guardrail Maintenance',
 'Fire Hydrant Snow Issue',
 'Fire Rig Visit Request',
 'Full TISH Report Request',
 'General Street Complaint',
 'Graffiti - Open311',
 'Graffiti complaint / reporting',
 'Gra

In [160]:
def prepare_311(df):
    df=df[df['TYPENAME'].isin(['Solid Waste Overflow Litter','Solid Waste Overflowing Litter - Open311','Commercial Food Safety/Sanitation Complaint'])]
    df=df[df.CASESTATUS==0]
    df['OPENEDDATETIME']=pd.to_datetime(df.OPENEDDATETIME)
    df['CLOSEDDATETIME']=pd.to_datetime(df.CLOSEDDATETIME)
    df=df.drop_duplicates()
    df=df[['CLOSEDDATETIME','OPENEDDATETIME','REASONNAME','TYPENAME','geometry']]
    df['case_duration']=df.CLOSEDDATETIME-df.OPENEDDATETIME
    return(df)

In [161]:
df=prepare_311(df)

In [115]:
df.to_pickle('311_data.pickle')